# Trabalho Prático 01 - Classificação KNN

A aplicação abaixa simula o algoritmo KNN (K-Neighrest-Neighbors). A passo a passo ele será detalhado.

## Aplicação

In [1]:
%pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd


1.7.1


In [7]:
# Lendo o arquivo CSV das flores.
dados_treinamento = pd.read_csv('Iris.csv')
print(dados_treinamento)

      Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
0      1            5.1           3.5            1.4           0.2   
1      2            4.9           3.0            1.4           0.2   
2      3            4.7           3.2            1.3           0.2   
3      4            4.6           3.1            1.5           0.2   
4      5            5.0           3.6            1.4           0.2   
..   ...            ...           ...            ...           ...   
145  146            6.7           3.0            5.2           2.3   
146  147            6.3           2.5            5.0           1.9   
147  148            6.5           3.0            5.2           2.0   
148  149            6.2           3.4            5.4           2.3   
149  150            5.9           3.0            5.1           1.8   

            Species  
0       Iris-setosa  
1       Iris-setosa  
2       Iris-setosa  
3       Iris-setosa  
4       Iris-setosa  
..              ...  
145  

In [ ]:
# Gerando os dados de teste
dados_teste = pd.DataFrame({
    "Id" :           [151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165],
    "SepalLengthCm": [5.1, 4.9, 5.8, 6.0, 5.5, 6.3, 6.8, 5.7, 7.1, 6.4, 5.0, 5.4, 6.2, 6.5, 5.9],
    "SepalWidthCm":  [3.5, 3.0, 2.7, 3.4, 3.0, 3.3, 3.0, 3.8, 3.0, 3.2, 3.6, 3.9, 2.8, 3.1, 3.0],
    "PetalLengthCm": [1.4, 1.5, 4.1, 4.5, 4.0, 4.7, 5.4, 1.7, 5.7, 4.6, 1.3, 1.5, 5.0, 5.2, 4.8],
    "PetalWidthCm":  [0.2, 0.3, 1.0, 1.5, 1.2, 1.6, 2.2, 0.4, 2.3, 1.5, 0.2, 0.2, 1.9, 2.0, 1.8],
    "Species":       [None]*15  # vazio, vai ser preenchido depois
    
})


k = [1,3,5,7]

import math

def distancia_euclidiana():
    resultado = []
    
    for _, row1 in dados_teste.iterrows():
        distancias = []
        for _,row2 in dados_treinamento.iterrows():
            d = math.sqrt((row1["SepalLengthCm"] - row2["SepalLengthCm"])**2+
                          (row1["SepalWidthCm"] - row2["SepalWidthCm"])**2+
                          (row1["PetalLengthCm"] - row2["PetalLengthCm"])**2+
                          (row1["PetalWidthCm"] - row2["PetalWidthCm"])**2
                          )
            distancias.append((d, row1.Id, row2.Id, row2.Species))
        resultado.append(distancias)
    
    return resultado


distancias = distancia_euclidiana()

for _, distancia in enumerate(distancias):
    distancia.sort(key=lambda x: x[0]);
    


[[(0.0, 151, 1, 'Iris-setosa'),
  (0.09999999999999998, 151, 18, 'Iris-setosa'),
  (0.1414213562373093, 151, 5, 'Iris-setosa'),
  (0.14142135623730964, 151, 40, 'Iris-setosa'),
  (0.14142135623730995, 151, 28, 'Iris-setosa'),
  (0.14142135623730995, 151, 29, 'Iris-setosa'),
  (0.17320508075688743, 151, 41, 'Iris-setosa'),
  (0.17320508075688762, 151, 8, 'Iris-setosa'),
  (0.22360679774997896, 151, 50, 'Iris-setosa'),
  (0.30000000000000016, 151, 22, 'Iris-setosa'),
  (0.30000000000000027, 151, 49, 'Iris-setosa'),
  (0.316227766016838, 151, 27, 'Iris-setosa'),
  (0.33166247903553986, 151, 20, 'Iris-setosa'),
  (0.3605551275463989, 151, 47, 'Iris-setosa'),
  (0.37416573867739383, 151, 36, 'Iris-setosa'),
  (0.3741657386773941, 151, 12, 'Iris-setosa'),
  (0.37416573867739483, 151, 11, 'Iris-setosa'),
  (0.3872983346207423, 151, 32, 'Iris-setosa'),
  (0.41231056256176635, 151, 37, 'Iris-setosa'),
  (0.4358898943540679, 151, 21, 'Iris-setosa'),
  (0.45825756949558394, 151, 44, 'Iris-setosa'